In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path

print(Path().cwd())
os.chdir(Path(os.getcwd()).parent)
print(Path().cwd())

/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS/CLI-Demos
/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS


## Select Contrast-Enhanced Ultrasound (CEUS) Cine and Parser

In [2]:
from src.image_loading.options import get_scan_loaders

print("Available scan loaders:", list(get_scan_loaders().keys()))

Available scan loaders: ['nifti']


In [ ]:
scan_type = 'nifti'

scan_path = '/media/das/TOSHIBA EXT/P-Selectin Data/July 2019 US imaging study Mice/July 2019 batch 3 imaging day 1/July2019mouse1imagingday1/20190725103303.756.nii.gz'
scan_loader_kwargs = {
    'transpose': False,
}

In [4]:
from src.entrypoints import scan_loading_step

image_data = scan_loading_step(scan_type, scan_path, **scan_loader_kwargs)

## Image Preprocessing

In [8]:
from src.image_preprocessing.options import get_im_preproc_funcs, get_required_im_preproc_kwargs

print("Available preprocessing functions:", list(get_im_preproc_funcs().keys()))

Available preprocessing functions: ['standardize', 'resample']


In [9]:
preproc_func_names = ['resample'] # in order of application
required_kwargs = get_required_im_preproc_kwargs(preproc_func_names)
print("Required kwargs for preprocessing functions:", required_kwargs)

Required kwargs for preprocessing functions: ['target_vox_size', 'interp']


In [10]:
preproc_kwargs = {
    'target_vox_size': (1.0, 1.0, 1.0),
    'interp': 'linear',
}

In [11]:
from src.entrypoints import scan_preprocessing_step

image_data = scan_preprocessing_step(preproc_func_names, image_data, **preproc_kwargs)

Resampling frames: 100%|██████████| 391/391 [00:02<00:00, 138.62it/s]


## Load Segmentation

Assumes same segmentation for each frame

In [12]:
from src.seg_loading.options import get_seg_loaders

print("Available segmentation loaders:", list(get_seg_loaders().keys()))

Available segmentation loaders: ['nifti_voi', 'load_bolus_mask']


In [ ]:
seg_type = 'nifti'

seg_path = '/media/das/TOSHIBA EXT/P-Selectin Data/July VOIs (Ashley)/20190725103303.756_segmentation_LEFT.nii.gz'
seg_loader_kwargs = {}

In [ ]:
from src.entrypoints import seg_loading_step

seg_data = seg_loading_step(seg_type, image_data, seg_path, scan_path, **seg_loader_kwargs)

## Segmentation Preprocessing

In [13]:
from src.seg_preprocessing.options import get_seg_preproc_funcs, get_required_seg_preproc_kwargs

print("Available preprocessing functions:", list(get_seg_preproc_funcs().keys()))

Available preprocessing functions: ['resample']


In [14]:
preproc_func_names = ['resample'] # in order of application
required_kwargs = get_required_seg_preproc_kwargs(preproc_func_names)
print("Required kwargs for preprocessing functions:", required_kwargs)

Required kwargs for preprocessing functions: ['target_vox_size', 'interp']


In [15]:
preproc_kwargs = {
    'target_vox_size': (1.0, 1.0, 1.0),
    'interp': 'nearest',
}

In [16]:
from src.entrypoints import seg_preprocessing_step

seg_data = seg_preprocessing_step(preproc_func_names, image_data, seg_data, **preproc_kwargs)

Resampling frames:   0%|          | 0/1 [00:00<?, ?it/s]

Resampling frames: 100%|██████████| 1/1 [00:00<00:00, 95.19it/s]


## CEUS Quantitative Temporal Curve Analysis

In [8]:
from src.time_series_analysis.options import get_analysis_types, get_required_kwargs

all_analysis_types, all_analysis_funcs = get_analysis_types()
print("Available analysis types:", list(all_analysis_types.keys()))

Available analysis types: ['curves', 'curves_paramap']


In [11]:
analysis_type = 'curves'

print("Available analysis functions:", list(all_analysis_funcs.keys()))

Available analysis functions: ['pyradiomics', 'tic']


In [ ]:
analysis_funcs = ['tic']

# Find all required kwargs for the analysis functions
analysis_funcs = analysis_funcs if len(analysis_funcs) else list(all_analysis_funcs[analysis_type].keys())
required_kwargs = get_required_kwargs(analysis_type, analysis_funcs)
print("Required kwargs for current analysis:", required_kwargs)

Required kwargs for current analysis: ['pyradiomics_config_paths', 'binwidth', 'min_intensity']


In [ ]:
analysis_kwargs = {
    'curves_output_path': 'sample.csv',
}

In [16]:
from src.entrypoints import analysis_step

analysis_obj = analysis_step(analysis_type, image_data, seg_data, analysis_funcs, **analysis_kwargs)

Computing curves: 100%|██████████| 391/391 [00:42<00:00,  9.17it/s]


## Curve Quantification

In [14]:
from src.curve_quantification.options import get_quantification_funcs

quantification_funcs = get_quantification_funcs()
print("Available quantification functions:", quantification_funcs.keys())

Available quantification functions: dict_keys(['auc_no_fit', 'cmus_firstorder', 'dte', 'first_order', 'lognormal_fit', 'wash_rates'])


In [15]:
function_names = ['dte'] # Empty list will use all functions
output_path = 'test_quants.csv'
curve_quantifications_kwargs = {
    'curves_to_fit': ['moderate-pselectin_diagnostics_Image-original_Mean'],
    'n_frames_to_analyze': 100,
    'tic_name': 'TIC'
}

In [16]:
from src.entrypoints import curve_quantification_step

curve_quant = curve_quantification_step(analysis_obj, function_names, output_path, **curve_quantifications_kwargs)